In [1]:
# 171805012 Refiye Şehnaz Yıldırım
# 171805043 Ersu Karpuz

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from nltk.corpus import stopwords
from re import search

In [3]:
#Upload Data
data = pd.read_excel("NLP1_train_tweets.xlsx",header=None)

data.columns = ['Sentence','Class']
data

,Sentence,Class
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz
...,...,...
13827,@Buragl111 turkcell kazık takıl sen bimcell le...,olumsuz
13828,@Turkcell merhaba turkcell.20 senelik dostlugu...,olumlu
13829,@TurkcellHizmet merhaba. Numarami bir baska op...,olumsuz
13830,iyi de Herkes TURKCELL Abonesi Değil ki ??,olumlu


In [4]:
#Text Preprocessing
columns = ['index','Class', 'Sentence']
df_ = pd.DataFrame(columns=columns)

#remove email adress
data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)

#remove IP address
data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)

#remove punctaitions and special chracters
data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')

#remove numbers
data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)

data["Class"].unique()

data['Sentence'] = data['Sentence'].str.strip().str.lower()

C:\Users\Ersu\AppData\Local\Temp/ipykernel_14452/4291951993.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')


# Negation

In [5]:
for i, sentence in enumerate(data["Sentence"]): 
    lst = sentence.split()
    substring = "değil"
    for fullstring in lst:
        if search(substring, fullstring):
            cümle = ""
            for j in lst:
                j = "NOT_" + j
                cümle = cümle + j + " "
            data["Sentence"].loc[i] = cümle
    substring = "yok"
    for fullstring in lst:
        if search(substring, fullstring):
            cümle = ""
            for j in lst:
                j = "NOT_" + j
                cümle = cümle + j + " "
            data["Sentence"].loc[i] = cümle

# Naive Bayes

In [6]:
for i, cell in enumerate(data["Class"]):
    if cell == "olumsuz":
        value = 0
    elif cell == "olumlu":
        value = 1
    else:
        value = 2
    data["Class"].loc[i] = value

In [8]:
x = data['Sentence']
y = data['Class']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [9]:
vec = CountVectorizer(stop_words=stopwords.words('turkish'))

x = vec.fit_transform(x).toarray()
x_test = vec.transform(x_test).toarray()

In [10]:
model = GaussianNB()
model.fit(x.astype('int'), y.astype('int'))

GaussianNB()

In [11]:
pred = model.predict(x_test)

In [12]:
tp = 0
fp = 0
tn = 0
fn = 0
for i in range(len(pred)):
    if pred[i] == 1 and y_test.values[i] == 1:
        tp += 1
    elif pred[i] == 0 and y_test.values[i] == 1:
        fp += 1
    elif pred[i] == 0 and y_test.values[i] == 0:
        tn += 1
    elif pred[i] == 1 and y_test.values[i] == 0:
        fn += 1

accuracy = (tp + tn)/(tp + fp + tn + fn)
print("accuracy =",accuracy*100)    

recall = (tp)/(tp+fn)
print("recall =",recall*100)

precision = tp/(tp+fp)
print("precision =",precision*100)

f1 = 2*(precision * recall)/(precision + recall)
print("f1 =",f1 *100)

accuracy = 64.22787738577213
recall = 79.08902691511386
precision = 55.096153846153854
f1 = 64.94757721734203
